In [1]:
import os
import glob
import pandas as pd

In [11]:
# concatenate the csv files.
# maybe just cat file1.csv file2.csv > outputfile.csv ...?
# todo: try and see what's fastest with many 'big' csv files
cwd = os.getcwd()
# for now...
os.chdir('/Users/geo/london_bikes/London-bikes-analysis/data')

files_names = [i for i in glob.glob('*.csv')]
print(files_names)

final_df = pd.concat([pd.read_csv(file) for file in files_names ])
# ../ just to avoid concatenating with final_csv by mistake when re-running this cell
final_df.to_csv( "../final_csv.csv", index=False, encoding='utf-8')

['157JourneyDataExtract10Apr2019-16Apr2019.csv', '158JourneyDataExtract17Apr2019-23Apr2019.csv']


In [12]:
print(final_df.head())
print(final_df.tail())
print(final_df.shape)

   Rental Id  Duration  Bike Id          End Date  EndStation Id  \
0   85830556      1080     2317  13/04/2019 14:25            553   
1   85824613       420    16173  13/04/2019 10:05            715   
2   85916752      1020    10848  16/04/2019 17:25            553   
3   85810735      1140     8626  12/04/2019 16:34            553   
4   85849238      1140     2139  14/04/2019 13:02            553   

              EndStation Name        Start Date  StartStation Id  \
0  Regent's Row , Haggerston   13/04/2019 14:07              835   
1     Aylward Street, Stepney  13/04/2019 09:58               33   
2  Regent's Row , Haggerston   16/04/2019 17:08              835   
3  Regent's Row , Haggerston   12/04/2019 16:15              835   
4  Regent's Row , Haggerston   14/04/2019 12:43              835   

            StartStation Name  
0  Farringdon Street, Holborn  
1      Central House, Aldgate  
2  Farringdon Street, Holborn  
3  Farringdon Street, Holborn  
4  Farringdon Street, 